In [3]:
import os
import findspark
home = os.path.expanduser("~")
findspark.init(os.path.join(home,"code/s_201511079","spark-2.0.0-bin-hadoop2.6"))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).config('spark.sql.warehouse.dir','file:///C:/Users/Eunda/code/s_201511079/data').getOrCreate()


jfile= os.path.join(os.environ["SPARK_HOME"],\
           "examples/src/main/resources/people.json")

_myDf= spark.read.json(jfile)
_myDf.filter(_myDf['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [4]:
import requests
r=requests.get("https://raw.githubusercontent.com/jokecamp/FootballData/master/World%20Cups/all-world-cup-players.json")
wc=r.json()

In [5]:
print type(wc), type(wc[0])

<type 'list'> <type 'dict'>


In [6]:
wc[0]

{u'Club': u'Club Atl\xc3\xa9tico Talleres de Remedios de Escalada',
 u'ClubCountry': u'Argentina',
 u'Competition': u'World Cup',
 u'DateOfBirth': u'1905-5-5',
 u'FullName': u'\xc3ngel Bossio',
 u'IsCaptain': False,
 u'Number': u'',
 u'Position': u'GK',
 u'Team': u'Argentina',
 u'Year': 1930}

In [7]:
wcDF=spark.createDataFrame(wc)
wcDF.printSchema()
wcDF.take(1)

C:\Users\Eunda\code/s_201511079\spark-2.0.0-bin-hadoop2.6\python\pyspark\sql\session.py:316: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)



[Row(Club=u'Club Atl\xc3\xa9tico Talleres de Remedios de Escalada', ClubCountry=u'Argentina', Competition=u'World Cup', DateOfBirth=u'1905-5-5', FullName=u'\xc3ngel Bossio', IsCaptain=False, Number=u'', Position=u'GK', Team=u'Argentina', Year=1930)]

In [8]:
from datetime import datetime
print datetime.strptime("11/25/1991", '%m/%d/%Y')

1991-11-25 00:00:00


In [16]:
wcRdd=spark.sparkContext.parallelize(wc)
wcRdd.take(1)

[{u'Club': u'Club Atl\xc3\xa9tico Talleres de Remedios de Escalada',
  u'ClubCountry': u'Argentina',
  u'Competition': u'World Cup',
  u'DateOfBirth': u'1905-5-5',
  u'FullName': u'\xc3ngel Bossio',
  u'IsCaptain': False,
  u'Number': u'',
  u'Position': u'GK',
  u'Team': u'Argentina',
  u'Year': 1930}]

In [18]:
from pyspark.sql.types import *
wcSchema=StructType([
    StructField("Club", StringType(), True),
    StructField("ClubCountry", StringType(), True),
    StructField("Competition", StringType(), True),
    StructField("DateOfBirth", DateType(), True),
    StructField("FullName", StringType(), True),
    StructField("IsCaptain", BooleanType(), True),
    StructField("Number", IntegerType(), True),
    StructField("Position", StringType(), True),
    StructField("Team", StringType(), True),
    StructField("Year", IntegerType(), True)
])

wcDF=spark.createDataFrame(wcRdd)
wcDF.printSchema()
wcDF.take(1)

root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)



[Row(Club=u'Club Atl\xc3\xa9tico Talleres de Remedios de Escalada', ClubCountry=u'Argentina', Competition=u'World Cup', DateOfBirth=u'1905-5-5', FullName=u'\xc3ngel Bossio', IsCaptain=False, Number=u'', Position=u'GK', Team=u'Argentina', Year=1930)]

In [19]:
wcDF=wcDF.withColumn('DoB', wcDF['DateOfBirth'].cast(DateType()))
wcDF=wcDF.withColumn('NumberInt', wcDF['Number'].cast("integer"))
wcDF.printSchema()
wcDF.take(1)

root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- DoB: date (nullable = true)
 |-- NumberInt: integer (nullable = true)



[Row(Club=u'Club Atl\xc3\xa9tico Talleres de Remedios de Escalada', ClubCountry=u'Argentina', Competition=u'World Cup', DateOfBirth=u'1905-5-5', FullName=u'\xc3ngel Bossio', IsCaptain=False, Number=u'', Position=u'GK', Team=u'Argentina', Year=1930, DoB=datetime.date(1905, 5, 5), NumberInt=None)]

In [30]:

from pyspark.sql import Row

myList=[('1','kim, js',170),('1','lee, sm', 175),('2','lim, yg',180),('2','lee',170)]
myRdd = spark.sparkContext.parallelize(myList)

In [32]:
_myRdd=myRdd.map(lambda x:Row(year=int(x[0]),name=x[1],height=int(x[2])))
_myDf=spark.createDataFrame(_myRdd)
_myDf.printSchema()
_myDf.take(1)

root
 |-- height: long (nullable = true)
 |-- name: string (nullable = true)
 |-- year: long (nullable = true)



[Row(height=170, name=u'kim, js', year=1)]

In [37]:
schema=StructType([
    StructField("age", IntegerType(), True),
    StructField("name", StringType(), True),
    #StructField("created", TimestampType(), True)
])
_myDf=spark.createDataFrame(_myRdd,schema)
_myDf.printSchema()
_myDf.show()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



Py4JJavaError: An error occurred while calling o515.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 21, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\Eunda\code\s_201511079\spark-2.0.0-bin-hadoop2.6\python\lib\pyspark.zip\pyspark\worker.py", line 172, in main
  File "C:\Users\Eunda\code\s_201511079\spark-2.0.0-bin-hadoop2.6\python\lib\pyspark.zip\pyspark\worker.py", line 167, in process
  File "C:\Users\Eunda\code\s_201511079\spark-2.0.0-bin-hadoop2.6\python\lib\pyspark.zip\pyspark\serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\Eunda\code/s_201511079\spark-2.0.0-bin-hadoop2.6\python\pyspark\sql\session.py", line 505, in prepare
    _verify_type(obj, schema)
  File "C:\Users\Eunda\code\s_201511079\spark-2.0.0-bin-hadoop2.6\python\lib\pyspark.z

In [33]:
_myDf.write.parquet(os.path.join("data","people.parquet"))
_pDf=spark.read.parquet(os.path.join("data","people.parquet"))
_pDf.show()

AnalysisException: u'path file:/C:/Users/Eunda/code/s_201511079/data/people.parquet already exists.;'

In [38]:
wcDF.printSchema()

root
 |-- Club: string (nullable = true)
 |-- ClubCountry: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- DateOfBirth: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- IsCaptain: boolean (nullable = true)
 |-- Number: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- DoB: date (nullable = true)
 |-- NumberInt: integer (nullable = true)



In [41]:
wcDF.createOrReplaceTempView("wc")
spark.sql("select Club,Team,Year from wc").show()

+--------------------+---------+----+
|                Club|     Team|Year|
+--------------------+---------+----+
|Club AtlÃ©tico Ta...|Argentina|1930|
|Quilmes AtlÃ©tico...|Argentina|1930|
|          Boca Junio|Argentina|1930|
|Central Norte TucumÃ|Argentina|1930|
|Club Atletico Est...|Argentina|1930|
|Racing Club de Av...|Argentina|1930|
|     Sportivo Barrac|Argentina|1930|
|          Boca Junio|Argentina|1930|
|Estudiantes de La...|Argentina|1930|
|Club AtlÃ©tico Sa...|Argentina|1930|
|          Boca Junio|Argentina|1930|
|Club Atletico Est...|Argentina|1930|
|Racing Club de Av...|Argentina|1930|
|Racing Club de Av...|Argentina|1930|
|Club Sportivo Bue...|Argentina|1930|
|Club AtlÃ©tico La...|Argentina|1930|
|Estudiantes de La...|Argentina|1930|
|Club AtlÃ©tico La...|Argentina|1930|
|Club AtlÃ©tico Hu...|Argentina|1930|
|          Boca Junio|Argentina|1930|
+--------------------+---------+----+
only showing top 20 rows



In [42]:
spark.catalog.listTables()

[Table(name=u'ed', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True),
 Table(name=u'wc', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True)]

In [43]:
wcDF.createOrReplaceTempView("wc")
wcPlayers=spark.sql("select FullName,Club,Team,Year from wc")

In [44]:
namesRdd=wcPlayers.rdd.map(lambda x: "Full name: "+x[0])
for e in namesRdd.take(5):
    print e

Full name: Ãngel Bossio
Full name: Juan Botasso
Full name: Roberto Cherro
Full name: Alberto Chividini
Full name: 


In [45]:
bucketDf=spark.createDataFrame([[1,["orange", "apple", "pineapple"]],
                                [2,["watermelon","apple","bananas"]]],
                               ["bucketId","items"])

In [46]:
bucketDf.show(truncate=False)

+--------+----------------------------+
|bucketId|items                       |
+--------+----------------------------+
|1       |[orange, apple, pineapple]  |
|2       |[watermelon, apple, bananas]|
+--------+----------------------------+



In [48]:
import os
import urllib
_url = 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz'
_fname = os.path.join(os.getcwd(),'data','kddcup.data_10_percent.gz')
if(not os.path.exists(_fname)):
    print "%s data does not exist! retrieving.." % _fname
    _f=urllib.urlretrieve(_url,_fname)


C:\Users\Eunda\code\s_201511079\data\kddcup.data_10_percent.gz data does not exist! retrieving..


In [49]:
_rdd = spark.sparkContext.textFile(_fname)

In [50]:
_rdd.count()

494021

In [53]:
_normal = _rdd.filter(lambda x: 'guess_passwd.' in x)
print _normal.count()

53


In [54]:

from pyspark.sql import Row

_csv = _rdd.map(lambda l: l.split(","))
_csvRdd = _csv.map(lambda p: 
    Row(
        duration=int(p[0]), 
        protocol=p[1],
        service=p[2],
        flag=p[3],
        src_bytes=int(p[4]),
        dst_bytes=int(p[5])
    )
)